#### Criando uma coluna que relaciona o ritmo da música com o gênero e a chance de ser hit

In [ ]:
Num_Muicas_porGenre_timesignature = df.groupby(['track_genre','time_signature']).size().reset_index(name = 'Num_Muicas_porGenre_timesignature')
Num_Muicas_porGenre_timesignature


Num_Hits_porGenre_timesignature = df[df['popularity_target'] == 1].groupby(['track_genre','time_signature']).size().reset_index(name = 'Num_Hits_porGenre_timesignature')


Num_Hits_porGenre_timesignature


# Mesclar as contagens de total de músicas e hits de volta ao DataFrame
df = df.merge(Num_Muicas_porGenre_timesignature, on=['track_genre', 'time_signature'], how='left')
df = df.merge(Num_Hits_porGenre_timesignature, on=['track_genre', 'time_signature'], how='left')
df['Num_Muicas_porGenre_timesignature'] = df['Num_Muicas_porGenre_timesignature'].fillna(0).astype(int)
df['Num_Hits_porGenre_timesignature'] = df['Num_Hits_porGenre_timesignature'].fillna(0).astype(int)

# Calcular a porcentagem de hits por intervalo
df['totalHit_%_perGenre_and_timesignature'] = (df['Num_Hits_porGenre_timesignature'] / df['Num_Muicas_porGenre_timesignature'] * 100).round(2)

df.drop(columns=['Num_Hits_porGenre_timesignature','Num_Muicas_porGenre_timesignature'],inplace=True)

#### Criando a coluna que relaciona o tempo da musica com porcentagem de hits nesse intervalo

In [ ]:
import pandas as pd

# Definir os intervalos (bins) para a duração em ms, incluindo três grupos
bins = [0, 150000, 240000, float('inf')]  # até 2:30, até 4:00, e acima de 4:00
labels = ['Até 2.30 min', 'Até 4 min', 'Acima de 4 min']

# Criar uma nova coluna com os intervalos de duração cumulativos
df['duration_range_cumulative'] = pd.cut(df['duration_ms'], bins=bins, labels=labels, include_lowest=True, right=True)

# Contar o número total de músicas por intervalo de duração cumulativo
musicas_por_intervalo = df.groupby('duration_range_cumulative').size().reset_index(name='total_musicas')

# Contar o número de hits por intervalo de duração cumulativo
hits_por_intervalo = df[df['popularity_target'] == 1].groupby('duration_range_cumulative').size().reset_index(name='total_hits')

# Mesclar as contagens de total de músicas e hits por intervalo de volta ao DataFrame
df = df.merge(musicas_por_intervalo, on='duration_range_cumulative', how='left')
df = df.merge(hits_por_intervalo, on='duration_range_cumulative', how='left')

# Preencher valores NaN com 0 (caso algum intervalo não tenha hits)
df['total_hits'] = df['total_hits'].fillna(0).astype(int)

# Calcular a porcentagem de hits por intervalo
df['totalHit_%_perDurationRange'] = (df['total_hits'] / df['total_musicas'] * 100).round(2)

# Verificar o resultado
df.drop(columns=["total_musicas",'total_hits'],inplace = True)


#### Criando a coluna que relaciona o genero pelo numero de hits nele

In [ ]:
hits_por_genero = df[df['popularity_target'] == 1].groupby('track_genre').size().reset_index(name= 'total_musicas_genero')

# Contar o número total de músicas por gênero
musicas_por_genero = df.groupby('track_genre').size().reset_index(name='total_musicas_genero')

# Contar o número de hits por gênero
hits_por_genero = df[df['popularity_target'] == 1].groupby('track_genre').size().reset_index(name='total_hits_genero')

# Mesclar as contagens de total de músicas e hits por gênero de volta ao DataFrame original
df = df.merge(musicas_por_genero, on='track_genre', how='left')
df = df.merge(hits_por_genero, on='track_genre', how='left')

df['Genero/HitEm%'] = (df['total_hits_genero'] / df['total_musicas_genero'] * 100).round(2)

## Colocando o modelo em produção fazendo as previsões

In [ ]:
df_teste = df_teste.merge(Num_Muicas_porGenre_timesignature, on=['track_genre', 'time_signature'], how='left')
df_teste = df_teste.merge(Num_Hits_porGenre_timesignature, on=['track_genre', 'time_signature'], how='left')

df_teste['Num_Muicas_porGenre_timesignature'] = df_teste['Num_Muicas_porGenre_timesignature'].fillna(0).astype(int)
df_teste['Num_Hits_porGenre_timesignature'] = df_teste['Num_Hits_porGenre_timesignature'].fillna(0).astype(int)

# Calcular a porcentagem de hits por intervalo
df_teste['totalHit_%_perGenre_and_timesignature'] = (df_teste['Num_Hits_porGenre_timesignature'] / df_teste['Num_Muicas_porGenre_timesignature'] * 100).round(2)
df_teste['totalHit_%_perGenre_and_timesignature'] = df_teste['totalHit_%_perGenre_and_timesignature'].fillna(0).astype(int)


df_teste.drop(columns=['Num_Hits_porGenre_timesignature','Num_Muicas_porGenre_timesignature'],inplace=True)

In [ ]:
# Verificar combinações únicas no treino e teste
combinacoes_treino = df[['track_genre', 'time_signature']].drop_duplicates()
combinacoes_teste = df_teste[['track_genre', 'time_signature']].drop_duplicates()

# Combinações no teste que não estão no treino
combinacoes_unicas_teste = combinacoes_teste[~combinacoes_teste.isin(combinacoes_treino.to_dict(orient='list')).all(axis=1)]
print(combinacoes_unicas_teste)


In [ ]:

# Transferir as métricas calculadas para o conjunto de teste
df_teste = df_teste.merge(musicas_por_genero, on='track_genre', how='left')
df_teste = df_teste.merge(hits_por_genero, on='track_genre', how='left')

df_teste['total_hits_genero'] = df_teste['total_hits_genero'].fillna(0).astype(int)
df_teste['total_musicas_genero'] = df_teste['total_musicas_genero'].fillna(0).astype(int)

df_teste['Genero/HitEm%'] = (df_teste['total_hits_genero'] / df_teste['total_musicas_genero'] * 100).round(2)

In [ ]:

df_teste['duration_range_cumulative'] = pd.cut(df_teste['duration_ms'], bins=bins, labels=labels, include_lowest=True, right=True)



df_teste = df_teste.merge(musicas_por_intervalo, on='duration_range_cumulative', how='left')
df_teste = df_teste.merge(hits_por_intervalo, on='duration_range_cumulative', how='left')

df_teste['total_hits'] = df_teste['total_hits'].fillna(0).astype(int)
df_teste['total_musicas'] = df_teste['total_musicas'].fillna(0).astype(int)

df_teste['totalHit_%_perDurationRange'] = (df_teste['total_hits'] / df_teste['total_musicas'] * 100).round(2)

In [ ]:

df_teste['Genero/HitEm%'].fillna(0, inplace=True)
df_teste['Energy_Genre'].fillna(df_teste['Energy_Genre'].mean(), inplace=True)  # se fizer sentido

In [ ]:
# Identificar as colunas novas que foram criadas e que contêm zeros a serem substituídos
colunas_novas = [
    'Genero/HitEm%', 'Energy_Genre', 'totalHit_%_perDurationRange', 
    'totalHit_%_perGenre_and_timesignature'
]

# Substituir os zeros pela mediana de cada coluna nova
for coluna in colunas_novas:
    mediana = df_teste[coluna].median()  # Calcular a mediana da coluna
    df_teste[coluna] = df_teste[coluna].replace(0, mediana)  # Substituir zeros pela mediana

# Verificar o resultado
print(df_teste[colunas_novas].head())